#### Ticketmaster Glue Job: Filling Null Numeric Values with 0s

#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [15]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 5f0144b8-2dd2-4822-9cfb-1441d957ec88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 5f0144b8-2dd2-4822-9cfb-1441d957ec88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 5f0144b8-2dd2-4822-9cfb-1441d957ec88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 5f0144b8-2dd2-4822-9cfb-1441d957ec88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



#### Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [16]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='tm-end-database', table_name='ticketmaster_data_csv')
dyf.printSchema()

root
|-- event_id: string
|-- event_name: string
|-- event_url: string
|-- event_date: string
|-- event_time: string
|-- event_genre: string
|-- event_subgenre: string
|-- min_price: double
|-- max_price: double
|-- ticket_limit: long
|-- venue_id: string
|-- venue_name: string
|-- venue_city: string
|-- venue_address: string
|-- venue_state: string
|-- venue_latitude: double
|-- venue_longitude: double
|-- venue_market: string
|-- main_attraction: string


#### Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [17]:
df = dyf.toDF()
df.show()

+-----------------+--------------------+--------------------+----------+----------+-----------+--------------+---------+---------+------------+------------+--------------------+---------------+--------------------+-------------+--------------+---------------+--------------------+-------------------+
|         event_id|          event_name|           event_url|event_date|event_time|event_genre|event_subgenre|min_price|max_price|ticket_limit|    venue_id|          venue_name|     venue_city|       venue_address|  venue_state|venue_latitude|venue_longitude|        venue_market|    main_attraction|
+-----------------+--------------------+--------------------+----------+----------+-----------+--------------+---------+---------+------------+------------+--------------------+---------------+--------------------+-------------+--------------+---------------+--------------------+-------------------+
|   vvG1bZ9MwOsvtU|P!NK: Summer Carn...|https://www.ticke...|2024-08-10|  18:30:00|       Rock|  

In [26]:
if df.head(1):
    df = df.fillna ({
        'min_price':0,
        'max_price':0,
        'ticket_limit':0
    })
    df.write.mode("overwrite").option("header", "true").csv("s3://ticketmaster-data/ticketmaster-export/imputed_csv/")

In [27]:
df_check = spark.read.option("header", "true").csv("s3://ticketmaster-data/ticketmaster-export/imputed_csv/")

In [28]:
df_check.show()

+-----------------+--------------------+--------------------+----------+----------+-----------+--------------+---------+---------+------------+------------+--------------------+---------------+--------------------+-------------+--------------+---------------+--------------------+-------------------+
|         event_id|          event_name|           event_url|event_date|event_time|event_genre|event_subgenre|min_price|max_price|ticket_limit|    venue_id|          venue_name|     venue_city|       venue_address|  venue_state|venue_latitude|venue_longitude|        venue_market|    main_attraction|
+-----------------+--------------------+--------------------+----------+----------+-----------+--------------+---------+---------+------------+------------+--------------------+---------------+--------------------+-------------+--------------+---------------+--------------------+-------------------+
|   vvG1bZ9MwOsvtU|P!NK: Summer Carn...|https://www.ticke...|2024-08-10|  18:30:00|       Rock|  

In [19]:
from awsglue.dynamicframe import DynamicFrame

In [23]:
dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")

In [24]:
glueContext.write_dynamic_frame.from_options(
    frame=dynamic_frame,
    connection_type="s3",
    connection_options={"path": "s3://ticketmaster-data/ticketmaster-export/imputed_csv/"},
    format="csv", 
    format_options={"withHeader": True}
)

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [25]:
s3output = glueContext.getSink(
  path="s3://ticketmaster-data/ticketmaster-export/imputed_csv/",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="tm-end-database", catalogTableName="ticketmaster_imputed_data"
)
s3output.setFormat("csv")
s3output.writeFrame(dynamic_frame)